In [8]:
# Python 3
%matplotlib inline

import axelrod as axl

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

plt.rcParams['figure.figsize'] = (10, 10)

In [9]:
first_tournament_participants_ordered_by_reported_rank = [s() for s in axl.axelrod_first_strategies]
number_of_strategies = len(first_tournament_participants_ordered_by_reported_rank)

for player in first_tournament_participants_ordered_by_reported_rank:
    print(player)

Tit For Tat
First by Tideman and Chieruzzi: (D, D)
First by Nydegger
First by Grofman
First by Shubik
First by Stein and Rapoport: 0.05: (D, D)
Grudger
First by Davis: 10
First by Graaskamp: 0.05
First by Downing
First by Feld: 1.0, 0.5, 200
First by Joss: 0.9
First by Tullock
First by Anonymous
Random: 0.5


In [10]:
tournament = axl.Tournament(players=first_tournament_participants_ordered_by_reported_rank, turns=100, repetitions=1000, seed=1)
results = tournament.play()

Analysing: 100%|██████████| 25/25 [00:03<00:00,  8.16it/s]


In [11]:
results.ranked_names

['First by Stein and Rapoport: 0.05: (D, D)',
 'Tit For Tat',
 'First by Shubik',
 'First by Grofman',
 'First by Tideman and Chieruzzi: (D, D)',
 'Grudger',
 'First by Davis: 10',
 'First by Nydegger',
 'First by Graaskamp: 0.05',
 'First by Downing',
 'First by Feld: 1.0, 0.5, 200',
 'First by Tullock',
 'First by Joss: 0.9',
 'First by Anonymous',
 'Random: 0.5']

In [12]:

# Assuming 'results' is your Axelrod tournament results
plot = axl.Plot(results)

# Extract data for boxplot
data = plot._boxplot_dataset  # Scores for each strategy
names = plot._boxplot_xticks_labels  # Names of the strategies
cooperation_ratings = [x.Cooperation_rating for x in results.summarise()]

# Normalize cooperation ratings to [0, 1]
min_rating = min(cooperation_ratings)
max_rating = max(cooperation_ratings)
normalized_ratings = [(x - min_rating) / (max_rating - min_rating) for x in cooperation_ratings]

# Retrieve the Viridis colorscale
viridis_colorscale = px.colors.get_colorscale("Viridis")

# Function to map a value to a color
def map_color(value, colorscale):
    # Assuming value is normalized (0 to 1)
    return colorscale[int(value * (len(colorscale) - 1))]

# Map the normalized values to colors
colors_mapped = [map_color(val, viridis_colorscale) for val in cooperation_ratings]

# Create a Plotly figure
fig = go.Figure()

fig.update_layout(autosize=False, width=550, height=500, margin=dict(l=5, r=10, b=10, t=50, pad=1))

# Add box plots for each dataset
for i, dataset in enumerate(data):
    # Map the normalized rating to the colorscale and extract the first element of the returned list
    color = colors_mapped[i][1]
    
    fig.add_trace(go.Box(
        y=dataset, 
        name=names[i],
        marker=dict(color=color),
        hovertemplate=f'{names[i]}<br>Median: {np.median(dataset):.2f}<br>Mean: {np.mean(dataset):.2f}<br>Min: {np.min(dataset):.2f}<br>Max: {np.max(dataset):.2f}<br>Cooperation Rating: {cooperation_ratings[i]:.2f}'
    ))

# Add title and format layout
fig.update_layout(
    title="Strategies from Axelrod's First Tournament",
    xaxis_title="Strategies",
    yaxis_title="Scores", 
    showlegend=False, 
    xaxis_visible=False, 
)

# Show the figure
fig.show()
fig.write_html("axelrod_first_tournament.html")

In [13]:
df_results = pd.DataFrame(results.summarise())

# Bin the cooperation ratings into 5 bins 
df_results['Cooperation_rating_bin'] = pd.cut(df_results['Cooperation_rating'], bins=5, labels=False)

df_results.head()

,Rank,Name,Median_score,Cooperation_rating,Wins,Initial_C_rate,CC_rate,CD_rate,DC_rate,DD_rate,CC_to_C_rate,CD_to_C_rate,DC_to_C_rate,DD_to_C_rate,Cooperation_rating_bin
0,0,"First by Stein and Rapoport: 0.05: (D, D)",2.659286,0.645344,10.0,1.0,0.597328,0.048016,0.128994,0.225661,0.979210,0.177428,0.593885,0.000000,2
1,1,Tit For Tat,2.590000,0.766320,0.0,1.0,0.688817,0.077503,0.073259,0.160421,1.000000,0.000000,1.000000,0.000000,3
2,2,First by Shubik,2.556429,0.670099,3.0,1.0,0.612259,0.057840,0.097961,0.231940,1.000000,0.000000,0.173564,0.263128,2
3,3,First by Grofman,2.555714,0.861491,1.0,1.0,0.726131,0.135361,0.061003,0.077506,1.000000,0.284491,0.284448,1.000000,3
4,4,"First by Tideman and Chieruzzi: (D, D)",2.530714,0.624494,11.0,1.0,0.601959,0.022534,0.087920,0.287586,0.993222,0.030775,0.372963,0.000000,2


In [14]:
# Aggregate scores by cooperation rating bin
df_results_grouped = df_results.groupby('Cooperation_rating_bin').agg({
    'Median_score': ['mean', 'std', 'count'], 
    'Cooperation_rating': ['mean']
})

df_results_grouped.columns = ['_'.join(col) for col in df_results_grouped.columns.values]

df_results_grouped

,Median_score_mean,Median_score_std,Median_score_count,Cooperation_rating_mean
Cooperation_rating_bin,,,,
0,2.082054,0.176110,4,0.394524
1,2.121607,0.453880,4,0.545184
2,2.547768,0.088387,4,0.659442
3,2.572857,0.024244,2,0.813906
4,2.502143,NaN,1,0.992024


In [15]:
fig = px.bar(df_results_grouped, x=df_results_grouped.index, y='Median_score_mean', error_y='Median_score_std', color='Cooperation_rating_mean', color_continuous_scale='Viridis')
fig.update_layout(autosize=False, width=550, height=500, margin=dict(l=5, r=10, b=10, t=50, pad=1))
fig.update_layout(
    title="Median Score by Binned Cooperation Rating",
    xaxis_title="Cooperation Rating Bins",
    yaxis_title="Median Score", 
    legend_title_text='Cooperation Rating',
    coloraxis_colorbar=dict(
        title="Cooperation Rating",
        tickvals=[0, 0.25, 0.5, 0.75, .99],
        ticktext=["0", "0.25", "0.5", "0.75", "0.99"],
    ),
    showlegend=False, 
    xaxis_visible=True, 
)
fig.show()

In [16]:
fig.write_html("cooperation_ratings.html")
